In [ ]:
import pandas as pd
df = pd.read_csv('cleardescription.csv')
df

In [ ]:
df['cleardescription']=df['cleardescription'].apply(lambda x: str(x))

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
for i in range(len(df.index)):
    l=[]
    words=df.loc[i]['cleardescription'].split()
    for word in words:
        a=morph.parse(word)[0].tag.POS
        if(a=='INFN'):a='VERB' #тут выравниваем обозначения
        if(a=='ADJF'):a='ADJ'
        st=str(word)+str('_')+str(a)
        l.append(st)
    new_words=" ".join(l)
    df.at[i,'newcleardescription']=new_words

In [ ]:
df.to_csv('newcleardescription.csv',index=False)

In [ ]:
import sys
import gensim
import zipfile
model_file = "180.zip"
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [ ]:
#Тут разные попытки посчитать схожесть и определить понимает ли обозначениея(можно выкинуть)
words = ['машина_NOUN','принимать_ADJ']
words2 = ['тягач_NOUN']

for word in words:
    if word in model:
        print(word)
        for i in model.most_similar(positive=[word], topn=10):
            print(i[0], i[1])
        print('\n')
    else:
        print(word + ' is not present in the model')
model.wmdistance(words,words2)

In [ ]:
# найдем похожие для фильма(считает  долго!)
d={}
l=[]
w= df.loc[0]['newcleardescription'].split() # для лестера(красота по американски)

for i in range(1,len(df.index)):
    k=model.wmdistance(w,df.loc[i]['newcleardescription'].split())
    d[i]=k
    list_d = list(d.items())
    list_d.sort(key=lambda i: i[1])
print(list_d[:15])
for i in range(len(list_d[:15])):
    print(df.loc[list_d[i][0]]['title'])